# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298659885095                   -0.85    5.0
  2   -8.300221806458       -2.81       -1.25    1.1
  3   -8.300448598715       -3.64       -1.89    3.6
  4   -8.300462436231       -4.86       -2.78    1.2
  5   -8.300464464683       -5.69       -3.08    3.8
  6   -8.300464546409       -7.09       -3.25    2.8
  7   -8.300464594633       -7.32       -3.40    3.6
  8   -8.300464618853       -7.62       -3.52    1.0
  9   -8.300464636957       -7.74       -3.71    1.0
 10   -8.300464641014       -8.39       -3.86    1.8
 11   -8.300464643538       -8.60       -4.11    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67464830660                   -0.70    6.6
  2   -16.67855941527       -2.41       -1.14    1.4
  3   -16.67923985421       -3.17       -1.86    4.2
  4   -16.67928158545       -4.38       -2.76    1.9
  5   -16.67928608229       -5.35       -3.20    5.1
  6   -16.67928621679       -6.87       -3.51    3.0
  7   -16.67928622244       -8.25       -3.97    1.0
  8   -16.67928622427       -8.74       -4.61    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32668195319                   -0.56    8.0
  2   -33.33444842448       -2.11       -1.00    1.2
  3   -33.33596383301       -2.82       -1.72    4.8
  4   -33.33685999230       -3.05       -2.34    6.6
  5   -33.33693847117       -4.11       -2.92    8.1
  6   -33.33694388948       -5.27       -3.84    3.2
  7   -33.33694392173       -7.49       -4.21    6.0


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298558135263                   -0.85    5.1
  2   -8.300272307499       -2.77       -1.59    1.0
  3   -8.300448885957       -3.75       -2.54    4.8
  4   -8.300369186640   +   -4.10       -2.27   10.5
  5   -8.300464522169       -4.02       -3.72    2.8
  6   -8.300464597644       -7.12       -3.97    4.4
  7   -8.300464643408       -7.34       -4.90    2.5


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32702633338                   -0.56    6.8
  2   -33.29472995074   +   -1.49       -1.26    1.2
  3   +4.478274550710   +    1.58       -0.28    8.4
  4   -33.30801725674        1.58       -1.55    6.6
  5   -33.24282231582   +   -1.19       -1.37    4.4
  6   -30.35708197418   +    0.46       -0.83    4.5
  7   -33.30440446260        0.47       -1.77    4.8
  8   -33.33492055790       -1.52       -2.30    3.1
  9   -33.33575954526       -3.08       -2.21    2.8
 10   -33.33625955117       -3.30       -2.60    2.2
 11   -33.33686578542       -3.22       -2.94    2.4
 12   -33.33688212649       -4.79       -3.01    2.8
 13   -33.33694190041       -4.22       -3.35    3.0
 14   -33.33694245581       -6.26       -3.78    3.4
 15   -33.33693689836   +   -5.26       -3.62    3.8
 16   -33.33694343747       -5.18       -4.15    3.1


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.